# Auto Correlation with TMC, CSP.LMC, and CBF in the Mid PSI
##### Last Updated: 20/03/24

This demo will show the basic operation of a 4-receptor 200MHz correlation driven from TMC to CSP.LMC and CBF. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

### Pre-requisites

This notebook has the following assumptions and pre-requisites:
* A deployment that includes TMC, CSP.LMC, and CBF is running. For example, one in a namespace `NS` that was launched from the [ska-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline.
* Taranta should be enabled in the deployment and the CSP.LMC Monitoring Dashboard can be used to monitor the relevant CSP.LMC and CBF devices.
* A virtual environment is being used. This notebook was made with Python 3.10 in mind.
* All requirements are installed via [poetry]().
* The talon boards `TARGET_TALONS` are powered off.

## Execution Steps

### Initial Parameter Set Up

The parameters below (except for `TANGO_HOST`) may need to be updated to match your deployment, environment, and paths. The config files are located within the ska-mid-psi repository in a subdirectory called `configs`.

NOTE: There are two hardware config files for the Mid PSI environment. `hw_config_psi.yaml` is used for Talons 1-4 and `hw_config_psi_swap.yaml` is used for Talons 5-8. Ensure the correct correct hardware config file is set in the parameters below.

In [ ]:
import os

print("Setting up parameters")

TARGET_TALONS = "1,2,3,4"
NS = "ci-ska-mid-psi-1218373306-maecst"
BITE_MAC_ADDRESS = "08:c0:eb:9d:47:78"

# TANGO HOST set up
CLUSTER_DOMAIN = "cluster.local"
TANGO_DB = "databaseds-tango-base"
TANGO_HOST = f"{TANGO_DB}.{NS}.svc.{CLUSTER_DOMAIN}:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

# Config files set up
COMMON_CONFIGS = "../configs/common"
CORR_CONFIGS = "../configs/4-receptor"
HW_CONFIG_FILE = f"{COMMON_CONFIGS}/hw_config/hw_config_psi.yaml"
DISH_CONFIG_FILE = f"{COMMON_CONFIGS}/sys_params/load_dish_config.json"
INIT_SYS_PARAM_FILE = f"{COMMON_CONFIGS}/sys_params/initial_system_param.json"
SLIM_CONFIG_FILE = f"{COMMON_CONFIGS}/slim_config/fs_slim_4vcc_1fsp.yaml"
ASSIGN_RESOURCES_FILE = f"{CORR_CONFIGS}/assign_resources_tmc.json"
CONFIGURE_SCAN_FILE = f"{CORR_CONFIGS}/configure_scan.json"
SCAN_FILE = f"{CORR_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{CORR_CONFIGS}/release_resources.json"

### Initial Set Up Using Engineering Console Deployer

The appropriate hardware config file is copied into the CBF Controller pod so that CBF knows how to map talon boards to LRUs to PDUs to power them on. 

Engineering Console Deployer is used to generate the talondx-config file, download the necessary artifacts from CAR, and configure the database.

In [ ]:
print("Copying the HW config file to the CBF Controller pod")
!kubectl cp {HW_CONFIG_FILE} {NS}/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

print(f"\nGenerating the talondx-config file for boards={TARGET_TALONS}")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards={TARGET_TALONS}

print("\nDownloading artifacts")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --download-artifacts

print("\nConfiguring the TANGO database")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --config-db

print("\nDONE")

### Set Up and Configure Device Proxies 

In [ ]:
from PyTango import DeviceProxy

print("Setting up the device proxies")

## TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")

## CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray= DeviceProxy("mid-csp/subarray/01")

## CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### Load the Dish Vcc Config / Init Sys Params

In [ ]:
with open(DISH_CONFIG_FILE) as f:
    dish_config_json = f.read()

tmc_central_node.LoadDishCfg(dish_config_json)

print(f"TMC CSP Master Dish Vcc Config: \n{tmc_csp_master.dishVccConfig}")
print(f"\nTMC CSP Master Source Dish Vcc Config: \n{tmc_csp_master.sourceDishVccConfig}")

with open(INIT_SYS_PARAM_FILE) as f:
    cbf_init_sys_params_json = f.read()

cbf_controller.InitSysParam(cbf_init_sys_params_json)

print(f"\nCBF Init Sys Params: \n{cbf_subarray.sysParam}")

### Configure CBF Controller with the SLIM config for 4-receptors

In [ ]:
print("Copying the appropriate SLIM config for 4-receptors into the CBF Controller pod")
!kubectl exec -ti -n {NS} ds-cbfcontroller-controller-0 -- /bin/bash -c "mkdir -p mnt/slim"
!kubectl cp {SLIM_CONFIG_FILE} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml

### Turn the Telescope On

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

print("Wait... ")

# Monitor the kubectl logs for the CBF Controller
!kubectl logs -n {NS} ds-cbfcontroller-controller-0 -f

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")

### Verify the Boards are On

SSH into each of the `TARGET_TALONS` to ensure they started correctly and run a "ps -ef" command on the board to ensure that the lower level device server executables have been copied over and are running.

### Generate the BITE Data using Engineering Console Bite

In [ ]:
print("Generating the BITE data")
!kubectl exec -ti -n {NS} ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards={TARGET_TALONS} --bite_mac_address={BITE_MAC_ADDRESS}

print("DONE")

### Monitor Visibilities Pod

These steps should be done in a separate terminal to monitor the packets when replaying the BITE data command a bit below.

In [ ]:
# kubectl apply -f viz_pod.yaml
# kubectl exec -ti sdn-dynamic-vis-1  -- bash
# apt update && apt install -y iproute2 tcpdump
# ip a 

# Update the IP in the Configure Scan to be the IP of the inet interface of the visibilities pod 

# tcpdump -i net1

### Assign Resources

In [ ]:
print("Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned")

with open(ASSIGN_RESOURCES_FILE) as f:
    assign_resources_json = f.read()

tmc_csp_subarray.AssignResources(assign_resources_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Configure Scan

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE) as f:
    configure_scan_json = f.read()

tmc_csp_subarray.Configure(configure_scan_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

### Replay the BITE Data using Engineering Console Bite

In [ ]:
print("Replay the BITE Data")

!kubectl exec -ti -n {NS} ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards={TARGET_TALONS}

print("DONE")

### Scan

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE) as f:
    scan_json = f.read()

tmc_csp_subarray.Scan(scan_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

### Clean Up


In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_csp_subarray.EndScan()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_csp_subarray.End()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print("Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty")

tmc_csp_subarray.ReleaseAllResources()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")